Reading data & creating an appropriate dataframe

In [7]:
import pandas as pd

movies_df = pd.read_csv('./resources/movies.csv')
ratings_df = pd.read_csv('./resources/ratings.csv')

df = pd.merge(ratings_df, movies_df, on='movieId', how='inner')
print(df.shape)
df.head()

(100836, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Let's narrow the data pool down to movies relevant enough to be rated atleast 100 times

In [17]:
agg_ratings = ratings_df.groupby('movieId').movieId.count().sort_values(ascending=False)
agg_ratings_gt100 = agg_ratings[agg_ratings.values > 100]
df_gt100 = df[df.movieId.isin(agg_ratings_gt100.index)]
print(df_gt100.shape)
df_gt100.head()

(19788, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Let's calculate movie similarities in the form of a matrix using Pearson's correlation

In [18]:
matrix = df_gt100.pivot(index='movieId',columns='userId', values='rating')
print(matrix.shape)
matrix.head()

(134, 597)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,3.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,5.0
10,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN
32,NaN,NaN,NaN,2.0,NaN,4.0,NaN,3.0,NaN,NaN,...,NaN,3.0,3.0,4.0,NaN,4.0,NaN,3.5,NaN,4.5


In [19]:
similar_movies = matrix.T.corr()
print(similar_movies.shape)
similar_movies.head()

(134, 134)


movieId,1,2,6,10,32,34,39,47,50,110,...,7153,7361,7438,8961,33794,48516,58559,60069,68954,79132
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.330978,0.106465,-0.021409,0.120087,0.407493,0.144193,0.222006,0.250781,0.028768,...,0.101284,0.049218,0.093972,0.643301,0.285349,0.326149,0.255290,0.237234,0.355424,0.076195
2,0.330978,1.000000,0.163510,0.016626,0.084588,0.330327,0.241754,0.088477,0.045364,0.311665,...,0.133258,0.164373,0.056481,0.460369,0.250228,0.274094,0.025349,-0.003548,0.296362,0.300878
6,0.106465,0.163510,1.000000,0.420222,0.184299,-0.005211,-0.169698,0.388253,0.201409,0.327672,...,0.088740,0.103805,-0.174723,0.074621,-0.004642,0.113802,0.143009,0.202412,0.164610,0.565437
10,-0.021409,0.016626,0.420222,1.000000,-0.160238,-0.281090,0.076431,0.243948,0.045645,0.060961,...,0.090872,0.346523,-0.041482,0.094992,0.162075,0.088284,0.152039,0.235220,-0.348508,-0.047223
32,0.120087,0.084588,0.184299,-0.160238,1.000000,0.238554,-0.308569,0.285574,0.255347,0.325520,...,0.312204,0.096976,0.421061,0.256342,0.217525,0.336010,0.350385,0.513187,0.512313,0.472037
